# Update Items

### Import Libraries

In [1]:
import glob
import os
import json
import xml.etree.ElementTree as ET
import requests
from urllib import parse
import argparse
import urllib.request
import configparser
import tqdm
import time

### Get configuration file

In [2]:
config = configparser.ConfigParser()
config.sections()
config.read('../config/api.ini')
client_id = config.get('main', 'client_id')
client_secret = config.get('main', 'client_secret')
endpoint = config.get('main', 'endpoint')

### Connect to the Api

In [3]:
params = {
    'key_identity': client_id,
    'key_credential': client_secret
}

access_url = endpoint
print("Connected to:    ", access_url)

Connected to:     http://localhost:8080/api/


### Navigate the xml and converts it to json

In [4]:
dictionary = {}
fileName = ''

for file in glob.glob("../input/*"):
    root = ET.parse(file).getroot()
    fileName = str(os.path.basename(file))
    
    for group in root.findall('family/group'):
        groupName = group.attrib['name']
        item = group.findall('item')
        
        for value in item:
            fieldName = value.attrib['name']         
            matchingName = groupName +"_"+ fieldName
            fieldValue = value.text
            
            #fix getty links 
            if "getty" in str(fieldValue) and 'subjectid=' in str(fieldValue) and '[' in str(fieldValue) and '[' in str(fieldValue):
                name  = fieldValue.split('[')[0].split(']')[0]
                url  = fieldValue.split('[')[1].split(']')[0]

                if  'subjectid=' in str(url):
                    url  = url.split('subjectid=')[1]
                    fieldValue = name + "; http://vocab.getty.edu/page/tgn/" +  url            

            cleanedText = str(fieldValue).replace("None", "").replace("\n", "").replace("\t", " ").replace('"', '').replace("'", "")
            
            # Creating/appending to a JSON structure
            if fileName not in dictionary:
                dictionary[fileName] = { matchingName :  [cleanedText] }
            else:
                if matchingName not in dictionary[fileName]:
                    dictionary[fileName][matchingName] = [cleanedText]
                else:
                    dictionary[fileName][matchingName].append(cleanedText)

#### Structure a JSON-LD item

In [109]:
def structureField(dict_type, property_id, value):
    return { dict_type: [ { "type": "literal", "property_id": property_id, "@value": value } ]}

def structureLabel(dict_type, property_id, value, **lang):
    
    if lang:
        return {"type": dict_type, "property_id": property_id, "@value": value, "@language": lang}
    else:
        return {"type": dict_type, "property_id": property_id, "@value": value}

### Import a single Item

In [6]:
item = 46 # The number according to the omeka s resource

Search for the identifier and the values in the XML File

In [7]:
url = endpoint + "items/" + str(item)
response = json.loads(urllib.request.urlopen(url).read().decode('utf-8'))
identifier = response['dcterms:identifier'][0]['@value']

print(identifier)

G_078_152


##### Simple entry

In [13]:
dcformat = structureField('dcterms:format', 9, dictionary[identifier]['Objekt_Gem\u00e4lde Ma\u00dfe max. (H x B x T in cm)'][0])
print(dcformat)

{'dcterms:format': [{'type': 'literal', 'property_id': 9, '@value': '75,7x57,6x2,5'}]}


##### Nested entries

In [8]:
valueID = 'Frühere Inventarnummer_Inventarnummer'
dictionaryID = 'dcterms:abstract'
propertyID = 19

In [9]:
# Check related entries - only for debug
for i in dictionary[identifier].keys():
    if valueID.split('_')[0] in i:
        singleItem = dictionary[identifier][i]
        print(i, singleItem)

Frühere Inventarnummer_Inventarnummer ['158', '663 (668?)', '78']
Frühere Inventarnummer_Herleitung ['Q3; Q5', 'B102; Q47', 'Q27']


In [11]:
objectEntry =  { dictionaryID: [] }

mainField = dictionary[identifier]['Frühere Inventarnummer_Inventarnummer']

# i am checking the length of the first element - double check if secondary fields have more entries
for i in range(len(mainField)): 
    
    itemid = mainField[i]
    note = dictionary[identifier]['Frühere Inventarnummer_Herleitung'][i]
     
    if(itemid != ""):
        elem = structureLabel('literal', propertyID, itemid)
        objectEntry[dictionaryID].append(elem)

    if(note != ""):
        elem = structureLabel('pinadatatypes:note', propertyID, note)
        objectEntry[dictionaryID].append(elem)   

print(objectEntry)

{'dcterms:abstract': [{'type': 'literal', 'property_id': 19, '@value': '158'}, {'type': 'pinadatatypes:note', 'property_id': 19, '@value': 'Q3; Q5'}, {'type': 'literal', 'property_id': 19, '@value': '663 (668?)'}, {'type': 'pinadatatypes:note', 'property_id': 19, '@value': 'B102; Q47'}, {'type': 'literal', 'property_id': 19, '@value': '78'}, {'type': 'pinadatatypes:note', 'property_id': 19, '@value': 'Q27'}]}


##### Merge all jsons previously defined

In [14]:
patch = { **dcformat, **objectEntry }

In [15]:
# Get current content
url = endpoint + "items/" + str(item)
response = json.loads(urllib.request.urlopen(url).read().decode('utf-8'))

identifier = response['dcterms:identifier'][0]['@value']

# Add items to post
response.update(patch)

# Merge with the response json
postRequest = json.dumps(response)

print(postRequest)

{"@context": "http://localhost:8080/api-context", "@id": "http://localhost:8080/api/items/46", "@type": ["o:Item", "dctype:Image"], "o:id": 46, "o:is_public": true, "o:owner": {"@id": "http://localhost:8080/api/users/1", "o:id": 1}, "o:resource_class": {"@id": "http://localhost:8080/api/resource_classes/26", "o:id": 26}, "o:resource_template": {"@id": "http://localhost:8080/api/resource_templates/2", "o:id": 2}, "o:thumbnail": null, "o:title": "G_078_152", "thumbnail_display_urls": {"large": null, "medium": null, "square": null}, "o:created": {"@value": "2021-04-06T09:35:39+00:00", "@type": "http://www.w3.org/2001/XMLSchema#dateTime"}, "o:modified": {"@value": "2021-04-08T14:21:36+00:00", "@type": "http://www.w3.org/2001/XMLSchema#dateTime"}, "o:media": [], "o:item_set": [], "o:site": [], "dcterms:identifier": [{"type": "literal", "property_id": 10, "property_label": "Identifier", "is_public": true, "@value": "G_078_152"}], "dcterms:abstract": [{"type": "literal", "property_id": 19, "@

#### Update fields

In [16]:
putUrl = endpoint + "items/" + str(item) 
headers = {"Content-Type": "application/json"}

print(putUrl)
r = requests.patch(putUrl, data=postRequest, params = params, headers=headers)

http://localhost:8080/api/items/46


### Upload Image

In [37]:
item = 46 # The number according to the omeka s resource

In [ ]:
image = 'test.png'
imageType = 'png'

In [ ]:
data = {
    "o:ingester": "upload", 
    "file_index": "0", 
    "o:item": {"o:id": item }
}

files = [
     ('data', (None, json.dumps(data), 'application/json')),
     ('file[0]', (image, open(image, 'rb'), 'image/' + imageType))
]

response = requests.post(endpoint + '/media', params=params, files=files)